# Libraries

In [2]:
import math
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from math import sqrt
import statsmodels
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from scipy.interpolate import interp1d
from statsmodels.tsa.vector_ar.var_model import VAR

# Load data

In [3]:
accounts_df = pd.read_csv("./data/accounts.csv")
transactions_df = pd.read_csv("./data/transactions.csv")

# Preprocessing

In [4]:
# Format Date
transactions_df.date=transactions_df.date.apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d'))
accounts_df.update_date=accounts_df.update_date.apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d'))

In [5]:
# Function to filter accounts that have less than 180 days of history
def valid_account_data(transactions, accounts):
    data_mod = transactions.copy()
    data_mod['valid'] = False
    for account in data_mod["account_id"].unique():
        if (accounts[accounts["id"]==account]["update_date"].item()-data_mod[data_mod["account_id"]==account]["date"].min()).days>180:
            data_mod.loc[data_mod["account_id"]==account,'valid'] = True
    data_mod = data_mod[data_mod["valid"]==True]
    data_mod = data_mod.reset_index(drop=True)
    data_mod.drop('valid', axis=1, inplace=True)
    return data_mod

In [6]:
# check accounts that have more than 180 days of history and discard the others 
valid_transactions_df = valid_account_data(transactions_df, accounts_df)
valid_accounts_df = accounts_df[accounts_df["id"].isin(valid_transactions_df["account_id"].unique())].reset_index(drop=True)

In [7]:
# Split positive and negative transactions
valid_transactions_df = valid_transactions_df.groupby(['account_id', "date"])['amount'].agg([('negative_trans' , lambda x : x[x < 0].sum()) , ('positive_trans' , lambda x : x[x > 0].sum())]).reset_index()

#### Aggregate transactions by week (168h)

In [11]:
weekly_negative_trs

In [12]:
# Account: 0
weekly_valid_transactions_df[weekly_valid_transactions_df["account_id"]==0].head()

,account_id,date,negative_trans,positive_trans
0,0,2020-10-13,-3446.71,3500.0
1,0,2020-10-20,-384.67,707.0
2,0,2020-10-27,-1170.48,945.0
3,0,2020-11-03,-349.14,200.0
4,0,2020-11-17,-102.00,101.0


### Filling missing dates

#### We have an Unevenly Spaced Time Series problem:
##### Some Time Series are not enough dense to interpolate the missing dates, and I'm not sure if I should introduce values in the dates which we are sure that don't have transactions.
#### I decide to introduce (-0.00) and (0.00) values in the missing dates until reach the update_date date

In [13]:
# fill missing dates
df_list = []
aux_df = weekly_valid_transactions_df.copy()
for group_name, group_df in aux_df.groupby(["account_id"]):
    idx = pd.date_range(group_df["date"].min(), pd.Timestamp("2021-07-04"), freq="168h")
    #if group_name==1246:
    #    print(idx)
    aux_df_2 = group_df.set_index(['date'])
    aux_df_2 = aux_df_2.reindex(idx, fill_value=-0.0)
    #if group_name==1246:
    #    print(aux_df_2.reset_index())
    aux_df_2["account_id"] = group_name
    df_list.append(aux_df_2.reset_index())
    
weekly_valid_transactions_df = pd.concat(df_list)
weekly_valid_transactions_df = weekly_valid_transactions_df.rename(columns={'index': 'date'})

weekly_valid_transactions_df.loc[weekly_valid_transactions_df['negative_trans'] == 0.00, 'negative_trans'] = -0.00
weekly_valid_transactions_df.loc[weekly_valid_transactions_df['positive_trans'] == -0.00, 'positive_trans'] = 0.00

In [14]:
weekly_valid_transactions_df

,date,account_id,negative_trans,positive_trans
0,2020-10-13,0,-3446.71,3500.0
1,2020-10-20,0,-384.67,707.0
2,2020-10-27,0,-1170.48,945.0
3,2020-11-03,0,-349.14,200.0
4,2020-11-10,0,-0.00,0.0
...,...,...,...,...
49,2021-06-01,1253,-0.00,0.0
50,2021-06-08,1253,-0.00,0.0
51,2021-06-15,1253,-0.00,0.0
52,2021-06-22,1253,-0.00,0.0


In [15]:
# Compute Total Transactions per date
weekly_valid_transactions_df["total_trans"] = weekly_valid_transactions_df["negative_trans"]+weekly_valid_transactions_df["positive_trans"]

In [16]:
# Compute intial balance for each account
valid_accounts_df['initial_balance'] = [round(valid_accounts_df[valid_accounts_df["id"]==account]["balance"].item()-weekly_valid_transactions_df[weekly_valid_transactions_df["account_id"]==account]["total_trans"].sum(),2) for account in valid_accounts_df["id"]]

In [17]:
valid_accounts_df

,id,balance,update_date,initial_balance
0,0,13.63,2021-07-02,15.84
1,1,12.91,2021-07-02,-89.97
2,4,2806.75,2021-07-02,20.00
3,7,0.38,2021-07-02,0.00
4,8,25.04,2021-07-02,-32.47
...,...,...,...,...
855,1249,5117.09,2021-07-02,0.00
856,1250,-2952.06,2021-07-02,215.63
857,1251,361.86,2021-07-02,265.00
858,1252,2074.71,2021-07-02,-0.00


In [18]:
# Compute balance of each entry in weekly_valid_transactions_df
weekly_valid_transactions_df['balance'] = 0
aux_df = weekly_valid_transactions_df.copy()
for group_name, group_df in aux_df.groupby(["account_id", "date"]):
    account = group_name[0]
    date = group_name[1]
    i_balance = valid_accounts_df[valid_accounts_df["id"]==account]["initial_balance"].item()
    if date == (weekly_valid_transactions_df[weekly_valid_transactions_df["account_id"]==account]["date"].min()):
        weekly_valid_transactions_df.loc[(weekly_valid_transactions_df.account_id==account) & (weekly_valid_transactions_df.date==date), 'balance'] = i_balance
    else:
        previous_data = weekly_valid_transactions_df[(weekly_valid_transactions_df["account_id"]==account) & (weekly_valid_transactions_df["date"]<date)]
        weekly_valid_transactions_df.loc[(weekly_valid_transactions_df.account_id==account) & (weekly_valid_transactions_df.date==date), 'balance'] = previous_data["total_trans"].sum()+i_balance

In [19]:
weekly_valid_transactions_df

,date,account_id,negative_trans,positive_trans,total_trans,balance
0,2020-10-13,0,-3446.71,3500.0,53.29,15.84
1,2020-10-20,0,-384.67,707.0,322.33,69.13
2,2020-10-27,0,-1170.48,945.0,-225.48,391.46
3,2020-11-03,0,-349.14,200.0,-149.14,165.98
4,2020-11-10,0,-0.00,0.0,0.00,16.84
...,...,...,...,...,...,...
49,2021-06-01,1253,-0.00,0.0,0.00,32.58
50,2021-06-08,1253,-0.00,0.0,0.00,32.58
51,2021-06-15,1253,-0.00,0.0,0.00,32.58
52,2021-06-22,1253,-0.00,0.0,0.00,32.58


# Preparing the Training and Test sets

#### I use the update_date as reference, so the training set will contain the trasactions of each account from its first transaction to the week 2021-06-07. The test set will be between 2021-06-07 and the update_date date

In [20]:
# create train test partition
weekly_valid_transactions_df = weekly_valid_transactions_df.set_index(weekly_valid_transactions_df['date'])

weekly_valid_transactions_train = weekly_valid_transactions_df[:'2021-06-07'].reset_index(drop=True)
weekly_valid_transactions_test  = weekly_valid_transactions_df['2021-06-07':].reset_index(drop=True)

/tmp/ipykernel_5159/686371609.py:5: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  weekly_valid_transactions_test  = weekly_valid_transactions_df['2021-06-07':].reset_index(drop=True)


In [23]:
# Train
weekly_valid_transactions_train

,date,account_id,negative_trans,positive_trans,total_trans,balance
0,2020-10-13,0,-3446.71,3500.0,53.29,15.84
1,2020-10-20,0,-384.67,707.0,322.33,69.13
2,2020-10-27,0,-1170.48,945.0,-225.48,391.46
3,2020-11-03,0,-349.14,200.0,-149.14,165.98
4,2020-11-10,0,-0.00,0.0,0.00,16.84
...,...,...,...,...,...,...
54959,2021-05-04,1253,-0.00,0.0,0.00,32.58
54960,2021-05-11,1253,-0.00,0.0,0.00,32.58
54961,2021-05-18,1253,-0.00,0.0,0.00,32.58
54962,2021-05-25,1253,-0.00,0.0,0.00,32.58


In [24]:
# Test
weekly_valid_transactions_test

,date,account_id,negative_trans,positive_trans,total_trans,balance
0,2021-06-08,0,-550.70,531.0,-19.70,46.32
1,2021-06-15,0,-25.80,0.0,-25.80,26.62
2,2021-06-22,0,-2634.29,2639.0,4.71,0.82
3,2021-06-29,0,-371.90,380.0,8.10,5.53
4,2021-06-08,1,-0.00,0.0,0.00,0.41
...,...,...,...,...,...,...
3435,2021-06-29,1252,-660.01,0.0,-660.01,2734.72
3436,2021-06-08,1253,-0.00,0.0,0.00,32.58
3437,2021-06-15,1253,-0.00,0.0,0.00,32.58
3438,2021-06-22,1253,-0.00,0.0,0.00,32.58


# Modelling: Vector Autoregression model

In [25]:
results = []
for account in weekly_valid_transactions_train["account_id"].unique():
    i_train= weekly_valid_transactions_train[weekly_valid_transactions_train["account_id"]==account][["date", "negative_trans","balance", "positive_trans"]]
    i_test = weekly_valid_transactions_test[weekly_valid_transactions_test["account_id"]==account][["date", "negative_trans","balance", "positive_trans"]]
    
    i_train = i_train.set_index(i_train['date'])
    i_train = i_train.drop(['date'], axis=1)
    i_test = i_test.set_index(i_test['date'])
    i_test = i_test.drop(['date'], axis=1)
    
    #fit the model
    model = VAR(endog=i_train)
    model_fit = model.fit()

    # make prediction on validation
    prediction = model_fit.forecast(model_fit.model.y,steps=len(i_test))
    
    res=pd.DataFrame({"Pred1":[x[0] for x in prediction], "Pred2":[x[1] for x in prediction], 
                      "Act1":i_test["negative_trans"].values, "Act2":i_test["balance"].values,
                     "account_id":account})
    results.append(res)

/home/harry/PycharmProjects/timeseries/venv/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency W-TUE will be used.
  warnings.warn('No frequency information was'
/tmp/ipykernel_5159/1432809.py:16: FutureWarning: y is a deprecated alias for endog, will be removed in version 0.11.0
  prediction = model_fit.forecast(model_fit.model.y,steps=len(i_test))
/home/harry/PycharmProjects/timeseries/venv/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency W-TUE will be used.
  warnings.warn('No frequency information was'
/tmp/ipykernel_5159/1432809.py:16: FutureWarning: y is a deprecated alias for endog, will be removed in version 0.11.0
  prediction = model_fit.forecast(model_fit.model.y,steps=len(i_test))
/home/harry/PycharmProjects/timeseries/venv/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:536: Val

In [35]:
mape_list = []
for df in results:
    m_predicted = df.Pred1.sum()
    m_actual = df.Act1.sum()
    
    mape_list.append((m_predicted-m_actual)/m_actual)

/tmp/ipykernel_5159/2143237817.py:12: RuntimeWarning: divide by zero encountered in double_scalars
  mape_list.append((m_predicted-m_actual)/m_actual)
/tmp/ipykernel_5159/2143237817.py:12: RuntimeWarning: invalid value encountered in double_scalars
  mape_list.append((m_predicted-m_actual)/m_actual)


# MdAPE

In [40]:
print("MdAPE: ",np.median(np.nan_to_num(mape_list, copy=True, nan=0.0, posinf=0.0, neginf=0.0)))

MdAPE:  0.06074649673147532


# MAPE

In [41]:
print("MAPE: ",np.mean(np.nan_to_num(mape_list, copy=True, nan=0.0, posinf=0.0, neginf=0.0)))

MAPE:  136.75215358753232
